In [1]:

from langfuse.callback import CallbackHandler
from dotenv import load_dotenv
import os
load_dotenv()

public_key = os.getenv("ENV_PUBLIC_KEY", None)
secret_key = os.getenv("ENV_SECRET_KEY", None)
env_host = os.getenv("ENV_HOST", None)
print(public_key, secret_key, env_host)

pk-lf-631b983e-b9d9-4eb9-873e-5e7a3cbfd772 sk-lf-7b566567-3011-4092-8fee-18cd47d4c65a http://localhost:3000


In [2]:
from langfuse import Langfuse
 
langfuse = Langfuse(public_key, secret_key, env_host, debug=True)

In [3]:
from langfuse.model import CreateDatasetRequest
 
langfuse.create_dataset(CreateDatasetRequest(name="gyliu-ds"))

Dataset(id='clnkmz9wg000ao709n7edh6n3', name='gyliu-ds', status=<DatasetStatus.ACTIVE: 'ACTIVE'>, project_id='clnkiwnyi0002o709jg28gtcv', created_at=datetime.datetime(2023, 10, 10, 18, 9, 20, 609000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2023, 10, 10, 18, 9, 20, 609000, tzinfo=datetime.timezone.utc), items=[], runs=[])

In [4]:
from langfuse.model import CreateDatasetItemRequest
 
langfuse.create_dataset_item(
    CreateDatasetItemRequest(
        dataset_name="gyliu-ds",
        # any python object or value
        input={
            "text": "hello world"
        },
        # any python object or value, optional
        expected_output={
            "text": "hello world"
        }
    )
)

ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/ycliu/lib/python3.10/site-packages/langfuse/client.py", line 143, in create_dataset_item
    return self.client.dataset_items.create(request=body)
  File "/Users/guangyaliu/ycliu/lib/python3.10/site-packages/langfuse/api/resources/dataset_items/client.py", line 68, in create
    raise NotFoundError(pydantic.parse_obj_as(str, _response.json()))  # type: ignore
  File "pydantic/tools.py", line 38, in pydantic.tools.parse_obj_as
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)


ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)